In [18]:
import pandas as pd

importer la base de donnée

In [19]:
data= pd.read_csv('C:/Users/HP/Desktop/mon_stage_senegal/data/Etudiants_2001_2024/Liste_globale_des_etudiants_tous_les_champs20232024.csv' , encoding='latin1' ,sep=';')


voir la les colonnes du fichier

In [20]:
data.columns

Index(['NUMERO', 'NUMERO_TABLE', 'INE', 'NUMERO_IDENTITIFICATION', 'NOM',
       'PRENOM', 'DATE_DE_NAISSANCE', 'LIEU_DE_NAISSANCE',
       'MAIL_INSTITUTIONNEL', 'SEXE', 'MENTION_BACC', 'ANNEE_BACC',
       'PAYS_DE_NAISSANCE', 'NATIONALITE', 'SERIE_BACC', 'REGION_DE_NAISSANCE',
       'ETABLISSMENT_CODE', 'ETABLISSEMENT', 'NIVEAU_SECTION',
       'SIGLE_NIVEAU_SECTION', 'COHORTE', 'ETAT_INSCRIPTION',
       'ANNEE_INSCRIPTION', 'ANNEE_UNIVERSITAIRE', 'TYPE_FORMATION',
       'CODE_NIVEAU', 'NIVEAU', 'DATE_INSCRIPTION', 'NIVEAU LMD',
       'NIVEAU LMD ET NON LMD', 'SYSTEME', 'DEPARTEMENT FORMATION'],
      dtype='object')

afficher les informations du dataframe

In [21]:
#fonction  les  informations du dataframe
def afficher_info(df):
    print("#==============Informations sur le DataFrame================ :")
    print(f"il y'a {df.shape[0]} lignes et {df.shape[1]} colonnes.")

afficher_info(data)
    

#==============Informations sur le DataFrame================ :
il y'a 88946 lignes et 32 colonnes.


on constate que tous les fichiers ont les meme colonnes avec des nombres de ignes differents 

reperons les colonnes avec des valeurs manquantes

In [22]:
#=========== Anomalie1 :  les valeurs manquantes dans les colonnes  ==========
def anomalie1(df):
    anomalie2_df = df.isnull().sum()
    anomalie2_df = anomalie2_df[anomalie2_df > 0]
    print(" le nombre de valeur manquante pour chaque colonne   :")
    print(anomalie2_df )
anomalie1(data) 

print(data['COHORTE'].unique())

 le nombre de valeur manquante pour chaque colonne   :
NUMERO_TABLE           37277
INE                    12993
MAIL_INSTITUTIONNEL       65
MENTION_BACC              20
ANNEE_BACC                20
SERIE_BACC              1265
REGION_DE_NAISSANCE     4602
COHORTE                75789
NIVEAU LMD             15719
dtype: int64
[nan 'A' 'B' 'C']


contrairement aux autre fichiers la colonne numero-d'identification n'as pas valeurs manquantes 
solution poissible pour gerer cette anomalie :

- solution possible :  contacter la disi pour regler le probleme ou sinon si une personne est inscrit avant 2010 avec un systeme lmd on le remplace par systeme classique et on essai de remplacer la colonne niveau_lmd_et_non_lmd par le niveau equivalent au syteme classique 
par exemple 
L1->1A
L2->2A 
L3->3A
M1->4A
M2->5A
D-> 6A !!!! peut etre 7A ou 8A a revoir


- on creer un nouveau dataframe sans les colonnes  numero_table et INE et meme numero d'identification 

-  contacter la disi pour remplir ces colonnes manquantes ou on essai de retrouver la serie bac a partir de depertaement de formation par exemple si le departement de formation a des suites de lettres comme med ou bio on remplace par S2 si l'année es posterier a 1998 ou D si le cas contraire 
si math ou phy  on rempalce par S1 si année  superieur 1998 ou C si  annee inferieur a 1998

-  contacter la disi pour le remplir ou peut etre  utiliser une api  comme OpenStreetMap  qui permet d'extraire la region a partir d'un lei de naissance(du texte) 

- creer un dataframe sans cette colonne (cohorte)

- lon peut garder que la colonne niveau_lmd_et_non_lmd 

contrairement aux autres fichiers la colonne numero d'dentifification n'as pas de valeur manquante  pour l'année 2024-2025
 

les années de naissance ou date de naissance incorrectes

In [43]:
#======anomalie3 :  les annéee de naissance pas bon ===============
def anomalie3(data):
    # Convertir la colonne 'DATE_DE_NAISSANCE' en datetime
    data['DATE_DE_NAISSANCE'] = pd.to_datetime(data['DATE_DE_NAISSANCE'], errors='coerce')
    # Extraire l'année de naissance
    data['annee_naissance'] = data['DATE_DE_NAISSANCE'].dt.year
    # Nettoyer et extraire uniquement les années de bac à 4 chiffres
    data['ANNEE_BACC'] = pd.to_numeric(
        data['ANNEE_BACC'].astype(str).str.extract(r'(\d{4})')[0], 
        errors='coerce'
    )
   # Calculer l'âge au bac uniquement pour les lignes valides
    data['age_bac'] = data['ANNEE_BACC'] - data['annee_naissance']
   # Rechercher les personnes ayant passé le bac avant 14 ans
    personnes_bac_avant_14_ans = data[data['age_bac'] < 14]
    personnes_bac_apres_40ans = data[data['age_bac'] > 40]
    personnes_bac_apres_30ans = data[data['age_bac'] > 30]
    print(f"Il y a {personnes_bac_avant_14_ans.shape[0]} personnes qui ont passé le bac avant 14 ans. voici leur année de naissance et l'année de bac : \t" )
    print(personnes_bac_avant_14_ans[['DATE_DE_NAISSANCE', 'ANNEE_BACC' ,'NUMERO' , 'age_bac']] )
    print(f"Il y a {personnes_bac_apres_40ans.shape[0]} personnes qui ont passé le bac apres 40 ans. voici leur année de naissance et l'année de bac : \t")
    print(personnes_bac_apres_40ans[['DATE_DE_NAISSANCE', 'ANNEE_BACC'  ,'NUMERO']])
    print(f"Il y a {personnes_bac_apres_30ans.shape[0]} personnes qui ont passé le bac avant 30 ans.! pas impossible mais rare")

    
anomalie3(data)     

Il y a 6 personnes qui ont passé le bac avant 14 ans. voici leur année de naissance et l'année de bac : 	
      DATE_DE_NAISSANCE  ANNEE_BACC      NUMERO  age_bac
14801        2003-01-02      2012.0  .20230C09S      9.0
19181        1993-10-11      2004.0  .20220BI1A     11.0
26491        1996-09-05      2004.0  .20220BIB5      8.0
39811        1990-06-03      2000.0  .200903Q1H     10.0
75711        1985-11-07      1985.0  .20210AU3H      0.0
86134        1983-06-06      1995.0  .20220C2S8     12.0
Il y a 25 personnes qui ont passé le bac apres 40 ans. voici leur année de naissance et l'année de bac : 	
      DATE_DE_NAISSANCE  ANNEE_BACC      NUMERO
2363         1969-10-06      2011.0  .200803C55
7431         1967-01-06      2010.0  .201004C3L
15035        1977-05-02      2019.0  .201909C10
19076        1979-04-08      2021.0  .20210ABXG
20272        1980-10-12      2022.0  .20220BIUN
20527        1981-01-02      2022.0  .20220BBZM
20784        1970-05-06      2017.0  .201708DLH
2181

La plupart des personnes ont obtenu leur bac la même année que leur naissance, ce qui constitue une incohérence

On peut conserver ces cas dans la base de données pour une autre usage mais pour notre cas on peut s'en passer

In [24]:
def anomalie4(data):
    # Compter les doublons exacts (lignes identiques)
    total = data.duplicated().sum()
    
    # Filtrer les étudiants apparaissant plusieurs fois avec le même numéro
    d = data[data.duplicated(subset=['NUMERO'], keep=False)]

    # Compter le nombre d'établissements différents pour chaque étudiant
    totale1 = d.groupby('NUMERO')['ETABLISSMENT_CODE'].nunique()
    totale2= d.groupby('NUMERO')['DEPARTEMENT FORMATION'].nunique() #meme numero avec des departement differentes
    #meme numero et meme etablissement mais niveaux differents
    # Regrouper par NUMERO et DEPARTEMENT, puis compter le nombre de niveaux distincts
    niveaux_par_departement = d.groupby(['NUMERO', 'DEPARTEMENT FORMATION'])['NIVEAU LMD ET NON LMD'].nunique().reset_index()

    # Garder uniquement ceux qui ont plus d’un niveau dans le même département
    niveaux_multiples = niveaux_par_departement[niveaux_par_departement['NIVEAU LMD ET NON LMD'] > 2]
    

    # Compter combien d'étudiants ont étudié dans plusieurs établissements
    nb_personnes_2_formations = (totale1 == 2).sum()
    nb_personnes_3_formations = (totale1 == 3).sum()
    nb_personnes_4_formations = (totale1 == 4).sum()
    nb_personnes_plus_4 = (totale1 > 4).sum()
    nb_personnes_inscrit_niveaux_differents = (totale2 == 2).sum()
    

    print(f"Il y a {total} doublons exacts dans le DataFrame.")
    print(f"Il y a {nb_personnes_2_formations} personnes qui ont étudié dans 2 établissements différents la meme année.")
    print(f"Il y a {nb_personnes_3_formations} personnes qui ont étudié dans 3 établissements différents.")
    print(f"Il y a {nb_personnes_4_formations} personnes qui ont étudié dans 4 établissements différents.")
    print(f"Il y a {nb_personnes_plus_4} personnes qui ont étudié dans plus de 4 établissements différents.")
    print(f"Il y a {nb_personnes_inscrit_niveaux_differents} personnes qui ont étudié dans la meme formation avec des niveaux differents.")
    print(f"Il y a {niveaux_multiples} personnes qui ont étudié dans le meme departement avec des niveaux differents.")
    
    
    
anomalie4(data)    


Il y a 0 doublons exacts dans le DataFrame.
Il y a 61 personnes qui ont étudié dans 2 établissements différents la meme année.
Il y a 0 personnes qui ont étudié dans 3 établissements différents.
Il y a 0 personnes qui ont étudié dans 4 établissements différents.
Il y a 0 personnes qui ont étudié dans plus de 4 établissements différents.
Il y a 235 personnes qui ont étudié dans la meme formation avec des niveaux differents.
Il y a          NUMERO                      DEPARTEMENT FORMATION  \
0    .199700AZC  Institut d'Odontologie et de Stomatologie   
77   .2010048TX     Chirurgie et Spécialités Chirurgicales   
152  .201305VIH  Médecine interne et Spécialités médicales   
155  .201305W4A  Médecine interne et Spécialités médicales   
531  .20200A83J     Chirurgie et Spécialités Chirurgicales   

     NIVEAU LMD ET NON LMD  
0                        3  
77                       3  
152                      3  
155                      3  
531                      3   personnes qui ont é

- 61 personnes sont inscrits dans deux etablissement(faclute instituts...) differents 
- 256 personnes sont inscrit dans la meme formation mais avec avec des niveaux diffent
- il y'a 5 personnes qui sont inscrit une meme formation avec 3 niveaux differents(anomalie)


256 personnes qui sont dans la meme formation mais dans des niveaux differents

In [25]:
#========== anomalie5 : les mentions au bac pas normales  ===========
def anomalie5(data):
    #les mentions au bac pas normales
    compteur = 0 
    mentions_normales  = ['AB', 'B', 'TB', 'PA']
    #compter le nombre de mentions anormales
    for i in data['MENTION_BACC']:
        if i not in mentions_normales:
            compteur+=1 
        
    print(f"il y'a {compteur} mentions anormales au bac")
    #afficher les mentions anormales
    mentions_anormales = data[~data['MENTION_BACC'].isin(mentions_normales)]['MENTION_BACC'].unique()
    print("les mentions anormales sont : ", mentions_anormales)
    
    
    
anomalie5(data)   

il y'a 3000 mentions anormales au bac
les mentions anormales sont :  ['BI' nan 'P' 'HO' 'TH' 'Da' '50']


il y'a 1848 mentions anormales au bac
les mentions anormales sont :  ['BI' 'P' 'TH'] 
solution possible : remplacer les serie avec la lettre B ou mb par BI la lettre p ou P devient PA et la lettre TH devient TB 
les autres comme 50 et 30 ka do pi .... on les remplacer.........(! a verifier plut tard) 

In [45]:
pd.set_option('display.max_columns', None)
data[data['MENTION_BACC']=="BI"]

,NUMERO,NUMERO_TABLE,INE,NUMERO_IDENTITIFICATION,NOM,PRENOM,DATE_DE_NAISSANCE,LIEU_DE_NAISSANCE,MAIL_INSTITUTIONNEL,SEXE,MENTION_BACC,ANNEE_BACC,PAYS_DE_NAISSANCE,NATIONALITE,SERIE_BACC,REGION_DE_NAISSANCE,ETABLISSMENT_CODE,ETABLISSEMENT,NIVEAU_SECTION,SIGLE_NIVEAU_SECTION,COHORTE,ETAT_INSCRIPTION,ANNEE_INSCRIPTION,ANNEE_UNIVERSITAIRE,TYPE_FORMATION,CODE_NIVEAU,NIVEAU,DATE_INSCRIPTION,NIVEAU LMD,NIVEAU LMD ET NON LMD,SYSTEME,DEPARTEMENT FORMATION,annee_naissance,age_bac
1,.20220AXN8,215446.0,NaN,.2762200300035,THIAM,Ndeye Rama,2003-04-01,DAKAR,ndeyerama1.thiam@ucad.edu.sn,F,BI,2022.0,Sénégal,Sénégalaise,S2,Dakar,ESP,Ecole Supérieure Polytechnique,Deuxième Année du Diplôme Universitaire de Tec...,Deuxième Année du Diplôme Universitaire de Tec...,NaN,Inscrit(e),2024,2023-2024,I,2,Deuxième Année,31/10/2023,NaN,2A,Classique,Génie Informatique,2003.0,19.0
4,.20230BJMT,230433.0,NaN,.2765200401256,NDAO,Khady,NaT,PIKINE,khady25.ndao@ucad.edu.sn,F,BI,2023.0,Sénégal,Sénégalaise,S2,Dakar,ESP,Ecole Supérieure Polytechnique,Licence 1 en Génie Biomédical,Licence 1 en Génie Biomédical,NaN,Inscrit(e),2024,2023-2024,C,1,Première Année,22/11/2023,L1,L1,LMD,Génie Electrique,NaN,NaN
26,.20230BXWE,63781.0,N02406720231,.1937201500348,DIALLO,Ibrahima,2004-09-07,DALIFORT,ibrahima351.diallo@ucad.edu.sn,M,BI,2023.0,Sénégal,Sénégalaise,S2,Dakar,FMPO,"Faculté de Médecine, de Pharmacie et d?Odonto-...",Licence 1 Médecine,Licence 1 Médecine,NaN,Inscrit(e),2024,2023-2024,I,1,Première Année,13/12/2023,NaN,1A,Classique,Médecine interne et Spécialités médicales,2004.0,19.0
54,.202009Q4C,NaN,N02332020201,.1584201300270,THIAM,Baye,NaT,KEUR MADIABEL,baye10.thiam@ucad.edu.sn,M,BI,2020.0,Sénégal,Sénégalaise,S2,Kaolack,FMPO,"Faculté de Médecine, de Pharmacie et d?Odonto-...",Master 1 Médecine,Master 1 Médecine,NaN,Inscrit(e),2024,2023-2024,I,4,Quatrième Année,01/04/2024,NaN,4A,Classique,Médecine interne et Spécialités médicales,NaN,NaN
170,.201708549,NaN,N00575F20172,.2425199901304,DJIGO,Safiétou,NaT,FATICK,safietou.djigo@ucad.edu.sn,F,BI,2017.0,Sénégal,Sénégalaise,S2,Fatick,FMPO,"Faculté de Médecine, de Pharmacie et d?Odonto-...",Doctorat 2 d'Exercice en Médecine,Doctorat 2 d'Exercice en Médecine,NaN,Inscrit(e),2024,2023-2024,I,7,Septième Année,22/03/2024,NaN,7A,Classique,Médecine interne et Spécialités médicales,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88893,.20220B0AB,171157.0,A06722220221,.1665200300766,DIAKHATE,Ndiaga,2003-10-07,NDIAKHATE,ndiaga4.diakhate@ucad.edu.sn,M,BI,2022.0,Sénégal,Sénégalaise,LAR,Thiès,FLSH,Faculté des Lettres et Sciences Humaines,Licence 2 Arabe,Licence 2 Arabe,NaN,Inscrit(e),2024,2023-2024,I,2,Deuxième Année,13/06/2024,L2,L2,LMD,Arabe,2003.0,19.0
88916,.20210AW9Q,NaN,NaN,.OP0763621,MBULU KINDA,Christian,1991-05-03,LUBUDI,christian.mbulukinda@ucad.edu.sn,M,BI,2011.0,Rép. Démo. Du Congo,Congolaise(RDC),S,NaN,FMPO,"Faculté de Médecine, de Pharmacie et d?Odonto-...",Troisième Année du D.E.S néphrologie,Troisième Année du D.E.S néphrologie,NaN,Inscrit(e),2024,2023-2024,C,11,Onzième année,13/05/2024,NaN,11A,Classique,Médecine interne et Spécialités médicales,1991.0,20.0
88929,.201708A1R,NaN,X0045A920172,.WM36373792,FAOUZI,Fatima Zahra,1999-06-11,RABAT,fatimazahra.faouzi@ucad.edu.sn,F,BI,2017.0,Maroc,Marocaine,S1,NaN,FMPO,"Faculté de Médecine, de Pharmacie et d?Odonto-...",Doctorat 2 d'Exercice en Médecine,Doctorat 2 d'Exercice en Médecine,NaN,Inscrit(e),2024,2023-2024,I,7,Septième Année,22/03/2024,NaN,7A,Classique,Médecine interne et Spécialités médicales,1999.0,18.0
88931,.201909B2W,NaN,N01725920192,.2752199901385,DIATTA,Moussouba,1999-06-11,DAKAR,moussouba.diatta@ucad.edu.sn,F,BI,2019.0,Sénégal,Sénégalaise,L2,Dakar,FLSH,Faculté des Lettres et Sciences Humaines,Master 1 Sociologie,Master 1 Sociologie,NaN,Inscrit(e),2024,2023-2024,I,4,Quatrième Année,05/09/2024,M1,M1,LMD,Sociologie,1999.0,20.0


il y'a 2962 personnes avec la mentions "BI"  on peut le remplacer par B 

In [27]:
pd.set_option('display.max_columns', None)
data[data['MENTION_BACC']=="P"]

,NUMERO,NUMERO_TABLE,INE,NUMERO_IDENTITIFICATION,NOM,PRENOM,DATE_DE_NAISSANCE,LIEU_DE_NAISSANCE,MAIL_INSTITUTIONNEL,SEXE,MENTION_BACC,ANNEE_BACC,PAYS_DE_NAISSANCE,NATIONALITE,SERIE_BACC,REGION_DE_NAISSANCE,ETABLISSMENT_CODE,ETABLISSEMENT,NIVEAU_SECTION,SIGLE_NIVEAU_SECTION,COHORTE,ETAT_INSCRIPTION,ANNEE_INSCRIPTION,ANNEE_UNIVERSITAIRE,TYPE_FORMATION,CODE_NIVEAU,NIVEAU,DATE_INSCRIPTION,NIVEAU LMD,NIVEAU LMD ET NON LMD,SYSTEME,DEPARTEMENT FORMATION,annee_naissance,age_bac
3524,.20230BJBI,NaN,NaN,.2619199602709,Fall,DEGUENE,1996-01-09,THIES,deguene9.fall@ucad.edu.sn,F,P,2018.0,Sénégal,Sénégalaise,S2,Thiès,ESP,Ecole Supérieure Polytechnique,Master 1 Genie Logiciel et Système d'Information,Master 1 Genie Logiciel et Système d'Information,NaN,Inscrit(e),2024,2023-2024,C,4,Quatrième Année,06/11/2023,M1,M1,LMD,Génie Informatique,1996.0,22.0
8376,.20230BJBV,NaN,NaN,.2452479,TRAORE,ALHASSANE ISSA,NaT,KAYES,alhassaneissa.traore@ucad.edu.sn,M,P,2016.0,Mali,Malienne,S2,NaN,ESP,Ecole Supérieure Polytechnique,Master 1 en Sces et Tech mention STIC spéciali...,Master 1 en Sces et Tech mention STIC spéciali...,NaN,Inscrit(e),2024,2023-2024,C,4,Quatrième Année,06/11/2023,M1,M1,LMD,Génie Informatique,NaN,NaN
22720,.20230BJBS,NaN,NaN,.AA0504117,DIALLO,AMARA,2003-09-10,MOPTI,amara.diallo@ucad.edu.sn,M,P,2020.0,Mali,Malienne,S,NaN,ESP,Ecole Supérieure Polytechnique,Master 1 en Sces et Tech mention STIC spéciali...,Master 1 en Sces et Tech mention STIC spéciali...,NaN,Inscrit(e),2024,2023-2024,C,4,Quatrième Année,06/11/2023,M1,M1,LMD,Génie Informatique,2003.0,17.0
30139,.20230BJCG,NaN,NaN,.RC4347960,MATTHIEU,BOGO,NaT,N'Djamena,bogo.matthieu@ucad.edu.sn,M,P,2023.0,Tchad,Tchadienne,S,NaN,ESP,Ecole Supérieure Polytechnique,Diplôme Superieur de Technologie Informatique 1,Diplôme Superieur de Technologie Informatique 1,NaN,Inscrit(e),2024,2023-2024,C,1,Première Année,07/11/2023,NaN,1A,Classique,Génie Informatique,NaN,NaN
50710,.20230BJBY,NaN,NaN,.21PP47090,AHOGA,THÉHILATEL MOYOSOLA LINDSEY IRMROSE,2001-08-09,Cheltenham,thehilatelmoyosola.ahoga@ucad.edu.sn,F,P,2018.0,Grande Bretagne,Béninoise,C,NaN,ESP,Ecole Supérieure Polytechnique,Master 1 en Sces et Tech mention STIC spéciali...,Master 1 en Sces et Tech mention STIC spéciali...,NaN,Inscrit(e),2024,2023-2024,C,4,Quatrième Année,06/11/2023,M1,M1,LMD,Génie Informatique,2001.0,17.0
53588,.20230BJCC,NaN,NaN,.1770199601808,GOMIS,JEAN FRÉDÉRIC SOBEL,NaT,RUFISQUE,jeanfredericsobel.gomis@ucad.edu.sn,M,P,2017.0,Sénégal,Sénégalaise,S2,Dakar,ESP,Ecole Supérieure Polytechnique,Master 1 en Sces et Tech mention STIC spéciali...,Master 1 en Sces et Tech mention STIC spéciali...,NaN,Inscrit(e),2024,2023-2024,I,4,Quatrième Année,07/11/2023,M1,M1,LMD,Génie Informatique,NaN,NaN
56071,.20230BJC0,NaN,NaN,.1757200000499,Thiam,CHEIKH MOUHAMED TIDIANE,2000-10-04,DAKAR,cheikhmouhamedtidiane.thiam@ucad.edu.sn,M,P,2019.0,Sénégal,Sénégalaise,S1,Dakar,ESP,Ecole Supérieure Polytechnique,Master 1 en Sces et Tech mention STIC spéciali...,Master 1 en Sces et Tech mention STIC spéciali...,NaN,Inscrit(e),2024,2023-2024,C,4,Quatrième Année,06/11/2023,M1,M1,LMD,Génie Informatique,2000.0,19.0
56205,.20230BJBW,NaN,NaN,.1052200401314,BA,ASSANE,1998-10-03,VELINGARA,assane37.ba@ucad.edu.sn,M,P,2016.0,Sénégal,Sénégalaise,S2,Kolda,ESP,Ecole Supérieure Polytechnique,Master 1 en Sces et Tech mention STIC spéciali...,Master 1 en Sces et Tech mention STIC spéciali...,NaN,Inscrit(e),2024,2023-2024,C,4,Quatrième Année,06/11/2023,M1,M1,LMD,Génie Informatique,1998.0,18.0
60249,.20230BJBU,NaN,NaN,.2751200002747,Dabo,Mame Coumba,NaT,DAKAR,mamecoumba.dabo@ucad.edu.sn,F,P,2018.0,Sénégal,Sénégalaise,S2,Dakar,ESP,Ecole Supérieure Polytechnique,Master 1 en Sces et Tech mention STIC spéciali...,Master 1 en Sces et Tech mention STIC spéciali...,NaN,Inscrit(e),2024,2023-2024,C,4,Quatrième Année,06/11/2023,M1,M1,LMD,Génie Informatique,NaN,NaN
68722,.20230BJBR,NaN,NaN,.23AL66677,DIALLO,MAMADOU,NaT,POINTE-NOIRE,mamadou273.diallo@ucad.edu.sn,M,P,2020.0,Congo (Brazzavil

il y'a 12 personnes avec la mentions "P"  on peut le remplacer par PA

In [28]:
pd.set_option('display.max_columns', None)
data[data['MENTION_BACC']=="TH"]

,NUMERO,NUMERO_TABLE,INE,NUMERO_IDENTITIFICATION,NOM,PRENOM,DATE_DE_NAISSANCE,LIEU_DE_NAISSANCE,MAIL_INSTITUTIONNEL,SEXE,MENTION_BACC,ANNEE_BACC,PAYS_DE_NAISSANCE,NATIONALITE,SERIE_BACC,REGION_DE_NAISSANCE,ETABLISSMENT_CODE,ETABLISSEMENT,NIVEAU_SECTION,SIGLE_NIVEAU_SECTION,COHORTE,ETAT_INSCRIPTION,ANNEE_INSCRIPTION,ANNEE_UNIVERSITAIRE,TYPE_FORMATION,CODE_NIVEAU,NIVEAU,DATE_INSCRIPTION,NIVEAU LMD,NIVEAU LMD ET NON LMD,SYSTEME,DEPARTEMENT FORMATION,annee_naissance,age_bac
41387,.201607UPB,NaN,X00B81820171,.AR5817937,BENFARES,ILYASS,NaT,OUED ZEM,ilyass.benfares@ucad.edu.sn,M,TH,2016.0,Maroc,Marocaine,S,NaN,FMPO,"Faculté de Médecine, de Pharmacie et d?Odonto-...",Doctorat 3 d'Exercice en Médecine,Doctorat 3 d'Exercice en Médecine,NaN,Inscrit(e),2024,2023-2024,I,8,Huitième année,04/04/2024,NaN,8A,Classique,Médecine interne et Spécialités médicales,NaN,NaN
45070,.201607UUF,NaN,X00B84220171,.GK8120408,ELMITARI,Haitam,1998-04-06,HAY HASSANI,haitam.elmitari@ucad.edu.sn,M,TH,2016.0,Maroc,Marocaine,S,NaN,FMPO,"Faculté de Médecine, de Pharmacie et d?Odonto-...",Doctorat 3 d'Exercice en Médecine,Doctorat 3 d'Exercice en Médecine,NaN,Inscrit(e),2024,2023-2024,I,8,Huitième année,04/04/2024,NaN,8A,Classique,Médecine interne et Spécialités médicales,1998.0,18.0
53900,.201004G3K,NaN,NaN,.2910199601978,NDIAYE,Adja Maguette,NaT,GUEDIAWAYE,adjamaguette.ndiaye@ucad.edu.sn,F,TH,2005.0,Sénégal,Sénégalaise,L'1,Dakar,ED-ARCIV,"Ecole doctorale Arts, Cultures et Civilisations",Doctorat Unique Etudes arabes et islamiques,Doctorat Unique Etudes arabes et islamiques,NaN,Inscrit(e),2024,2023-2024,I,6,Sixième Année,01/07/2024,Doctorat,Doctorat,LMD,Arabe,NaN,NaN


3 personnes avec la mention TH qui veut probablement dire trees bien TR

In [29]:
#==========anomalie6 : les series de bac pas valides =======
def anomalie6(data):
    #les series de bac pas valides  
    compteur = 0 
    compteur1 = 0
    series_normales_avant_98 = [ "A1", "A2", "A3", "A4",  "C", "D", "E","F1", "F2", "F3", "F4", "F5", "F6", "F7", "G1", "G2", "G3",  "T1", "T2",  "L'", "S'"  ]  
    series_normales_apres_98 = ["L1B", "S1A", "S1", "L1A", "S3", "S4", "T1", "STEG", "SEA", "F6","S5", "S2", "STIDD", "LA", "T2", "L-AR", "L2", "L'1"]         
     #convertir la colonne 'ANNEE_BACC' en entier 
    data['ANNEE_BACC'] = pd.to_numeric(data['ANNEE_BACC'], errors='coerce')
    #compter le nombre de series anormales
    for _, i in data.iterrows():
        if  i['NATIONALITE'] =="Sénégalaise" and i['SERIE_BACC'] not in series_normales_avant_98 and i['ANNEE_BACC'] < 1998:
            compteur+=1 
        if   i['NATIONALITE'] =="Sénégalaise" and i['SERIE_BACC'] not in series_normales_apres_98 and i['ANNEE_BACC'] >= 1998:
            compteur1+=1    
        
    print(f"il y'a {compteur} qui ont passe leur bac avant  1998 qui une serie de bac pas normale")
    #afficher les series anormales
    series_anormales = data[~data['SERIE_BACC'].isin(series_normales_avant_98) & (data['ANNEE_BACC'] < 1998)]['SERIE_BACC'].unique()
    print("ces series sont : ", series_anormales)
    print(f"il y'a {compteur1} qui ont passe leur bac apres  1998 qui une serie de bac pas normale")
    #afficher les series anormales
    series_anormales1 = data[~data['SERIE_BACC'].isin(series_normales_apres_98) & (data['ANNEE_BACC'] >= 1998)]['SERIE_BACC'].unique()
    print("ces series sont : ", series_anormales1)
    
     
anomalie6(data)    

il y'a 108 qui ont passe leur bac avant  1998 qui une serie de bac pas normale
ces series sont :  ['L1A' 'S' nan 'S2' 'L' 'L2' 'B' 'A' 'LAR' 'SM' 'SNA' "L'1" 'STEG' 'a3'
 'S1']
il y'a 3916 qui ont passe leur bac apres  1998 qui une serie de bac pas normale
ces series sont :  ['D' 'S' nan 'LAR' 'A' 'A4' 'C' 'S2A' 'G' 'A3' 'SNA' 'L' 'SNB' "L'" 'SB'
 'BT' 'SM' 'CAEP' 'ES' 'G2' 'F2' 'A2' 'B' 'T' 'E' 'F7' 'a3']


il y'a 108 senegalais qui ont passe leur bac avant  1998 qui une serie de bac pas normale
ces series sont :  ['B' 'S2' 'S' nan 'A' 'L2']
il y'a 3916 senegalais qui ont passe leur bac apres  1998 qui une serie de bac pas normale
ces series sont :  ['S' nan 'LAR' 'D' 'S2A' 'A4' 'BT' 'A3' 'A' 'C' 'G' 'SM' 'SNA' 'F7' 'L'
 "L'" 'T' 'B' 'SNB' 'ES' 'G2' 'SB' 'F2' 'E' 'A2']

 solution possible : meme raisonnement pour les valeurs manquantes on vas essayer de deviner les mauvaises series renseignes en fonction du departement de formation et de l'anne d'inscription 